<a href="https://colab.research.google.com/github/joseph-owiti/uva-mcs-general/blob/develop/Okeno_Storms_NLP_HW02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Submission Deadline: __October 6, 2024, 11:59 PM__

A penalty will be applied for late submission. Please refer to the course policy for more detail.  

## Instructions

Please read the instructions carefully before you start working on the homework.

- Please follow instructions and printed out the results as required. Keep the printed results and your implementation for grading purpose.
    - The TAs will not run your code for grading purpose unless it is necessary. That means, you may lose some points if the printed results are not in the submitted file.
- Submission should be via Canvas.
    - If you use Google Colab for running the code, please download the file and submit it via Canvas once it's done.
    - Submission via a Google Colab link will be considered as an invalid submission.
- Please double check the submitted file once you upload it to Canvas.
    - Students should be responsible for checking whether they submit the right files.
    - Re-submission is not allowed once the deadline is passed.

Also, if you missed the class lectures, please study the course materials first before working on the homework. It may save you some time.

# Homework 02 Word Embeddings

### Goal

The **goal** of this homework is to provide an opportunity to build an end-to-end system.

Specifically, we are going to build a word embedding system, that can

1. Read and preprocess raw data
2. Use two different ways (latent semantic analysis and skip-gram) to learn word embeddings
3. Evaluate the quality of word embeddings using some intrinsic evaluation methods

### Submission

Your submission should only include this notebook file. Please keep **all the outputs** in your submission for grading. We will run the code only if we are not sure it is correct.

### Dependency

You will need the following package to finish this homework assignment

- [spaCy](https://pypi.org/project/spacy/)
- [fasttext](https://pypi.org/project/fasttext/)

### Hint

Search for the keyword `TODO` to find out which parts need your input

In [2]:
# Download the data from course webpage
import urllib.request
from os.path import isfile
if not isfile("embeddings/imdb-small.txt"):
    url = "https://yangfengji.net/uva-nlp-grad/data/embeddings.zip"
    print("Downloading ...")
    filename, headers = urllib.request.urlretrieve(url, filename="embeddings.zip")

    print("Decompressing the file ...")
    !unzip embeddings.zip

sents = open("embeddings/imdb-small.txt").read().split("\n")
print("Read {} sentences".format(len(sents)))

Read 10000 sentences


## 1. Data Processing (5 points)

Data processing is an **essential** skill for NLP researchers. Unlike machine learning where researchers sometimes may want to use synthetic data to demonstrate the potential of their algorithms, NLP researchers need to deal with real-world data all the time. Unfortunately, this means that these data are noisy and often contain irregular patterns. Therefore, a reasonable data processing can alleviate the challenge of building NLP systems to some extent and may also help boost the performance of machine learning models.

Data processing for learning word embeddings includes two basic modules

- Tokenizing texts and replacing some special tokens
- Filtering low-frequency and building a vocab

### 1.1 Tokenization (2 points)

The following function *tokenize()* should include the following components

1. Load the raw text from the file named **imdb-small.txt**
2. Convert all characters into lower cases
3. Tokenize the raw text using `spaCy`
4. Remove all punctuation (as single tokens) and replace all numbers (as single tokens) with a special token `<num>`
5. Write the preprocessed text to the file named **imdb-small.txt.tokenized** and maintain the same format (one paragraph per line)

(The file names are pre-defined, please do not change them.)

In [3]:
# TODO: add necessary packages here
from spacy.lang.en import English
import re
import string

def tokenize(infname="embeddings/imdb-small.txt"):
    outfname = open(infname + ".tokenized", "w")

    # ----------------------------------------
    # TODO: add your code here

    with open(infname, 'r') as f:
        text = f.read()

    text = text.lower()
    paragraphs = text.split('\n')

    nlp = English()
    tokenize = nlp.tokenizer

    for paragraph in paragraphs:
        if paragraph.strip():
            tokens = [token.text for token in tokenize(paragraph)]

            # Clean up tokens
            new_tokens = []
            for token in tokens:
                if token.isalpha():
                    new_tokens.append(token)
                elif token.isdigit():
                    new_tokens.append("<num>")
                elif token in string.punctuation:
                    continue
                else:
                    new_tokens.append(token)

            outfname.write(" ".join(new_tokens) + "\n")

    # ----------------------------------------

    outfname.close()


### 1.2 Filtering (2 points)

The following function *token_filter()* should include the following components

1. Remove the words that appear in the data less than 5 times (word_frequency < 5)
2. Write the filtered data to the file named **imdb-small.txt.filtered** and maintain the same format (one sentence per line)
3. Return a Python list that contains all the words

In [4]:
# TODO: add necessary packages here
from collections import Counter

def token_filter(infname="embeddings/imdb-small.txt.tokenized", thresh=5):
    outfname = open(infname.replace(".tokenized", ".filtered"), 'w')
    vocab = []

    # ----------------------------------------
    # TODO: remove "pass" and add your code here

    with open(infname, 'r') as f:
        text = f.read()
    paragraphs = text.split('\n')

    all_words = [word for paragraph in paragraphs for word in paragraph.split()]
    word_counts = Counter(all_words)
    print("Original vocab size = %d" % len(word_counts))

    filtered_vocab = {word for word, count in word_counts.items() if count > thresh}

    for paragraph in paragraphs:
        new_paragraph = " ".join([word for word in paragraph.split() if word in filtered_vocab])
        outfname.write(new_paragraph + "\n")  # Write the filtered paragraph to the output file

    # ---------------------------------------

    outfname.close()
    return list(filtered_vocab)

### 1.3 Put all together (1 point)

The following code block will call the previous two functions to do data preprocessing.

This code block should include the following steps

- tokenization
- build the vocabulary with the variable name `vocab`
- print out the size of the vocabulary

In [5]:
tokenize()
vocab = token_filter()
print("The final vocab size = {}".format(len(vocab)))

Original vocab size = 57264
The final vocab size = 16382


## 2. Word Embeddings (5 points)

In this section, you need to implement two different ways of constructing word embeddings: latent semantic analysis  and skipgram.

### 2.1 Latent semantic analysis (3 points)

The function of LSA should include the following components

- Construct the word-doc matrix using `CountVectorizer` with `tokenizer=lambda x : x.split()`, make sure in this matrix that each row represents one word and each column represents one document (sentence, to be accurate in this case)
- Use the `TruncatedSVD` from `sklearn.decomposition` to factorize the word-doc matrix
- Construct the word embedding matrix with dimensionality as $v \times k$, where $v$ is the vocab size and $k$ is the word embedding dimension

The LSA() function should return

- **embeddings**: A matrix with size $v\times k$ that contains all the word embeddings
- **vocab**: A Python dict with size $v$ that maps a word to the corresponding word index. Please pay attention to the mapping relation in vocab, which will be needed in the evaluation section.

In [6]:
# TODO: add necessary packages here
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy

def LSA(fname = "embeddings/imdb-small.txt.filtered", dim=50):
    sents = open(fname).read().split("\n")
    print("Read {} sentences".format(len(sents)))

    # -------------------------------------
    # TODO: add your code here

    vectorizer = CountVectorizer(tokenizer=lambda x : x.split())
    vectorizer.fit(sents)
    vocab = vectorizer.vocabulary_

    word_doc_matrix = vectorizer.transform(sents)

    svd = TruncatedSVD(n_components=dim)
    embeddings = svd.fit_transform(numpy.transpose(word_doc_matrix))

    # -------------------------------------
    return embeddings, vocab


### 2.2 Skip-gram model (2 points)

In this section, you do not have to implement the skip-gram model by yourself. An authentic implementation of skip-gram can be found in the Python package [fasttext](https://pypi.org/project/fasttext/), which you can install on the your local machine with the folllwing commandline or directly load the package if you are using Google Colab.
```python
pip install fasttext
```

In the following code, please use the `fasttext.train_unsupervised` function for the skipgram() implementation. For the `fasttext.train_unsupervised`, please use the following configurations

- `model='skipgram'`
- Context window size: `ws = 3`
- Word embedding dimension: `dim = 50`
- Number of negative examples: `neg = 5`

For all other parameters, use their default values.

Similar to the previous LSA(), Skipgram() should return

- **embeddings**: A matrix with size $v\times k$ that contains all the word embeddings
- **vocab**: A Python dict with size $v$ that maps an index to the corresponding word

To get the word embeddings and vocab from fasttext, you need to understand [some functions](https://pypi.org/project/fasttext/#api) provided by the `model` object in the fasttext.

In [7]:
!pip install fasttext

In [9]:
# TODO: add necessary packages here
import fasttext

def Skipgram(fname = "embeddings/imdb-small.txt.filtered", ws=3, dim=50, neg=5):
    # ------------------------------------------
    # TODO: add your code here

    model = fasttext.train_unsupervised(str(fname), model='skipgram', ws=ws, dim=dim, neg=neg)

    embeddings = model.get_output_matrix()
    vocab = model.get_words()

    vocab = {word: index for index, word in enumerate(vocab)}

    # ------------------------------------------
    return embeddings, vocab


### 2.3 Put all together

Run the following code blocks to get word embeddings from two different methods. It may take a couple of minutes to compute both embeddings.

In [10]:
embeddings_lsa, vocab_lsa = LSA()
print("The LSA embedding matrix shape = {}".format(embeddings_lsa.shape))
print("The LSA vocab size = {}".format(len(vocab_lsa)))
embeddings_sg, vocab_sg = Skipgram()
print("The Skipgram embedding matrix shape = {}".format(embeddings_sg.shape))
print("The Skipgram vocab size = {}".format(len(vocab_sg)))

Read 10001 sentences
The LSA embedding matrix shape = (16382, 50)
The LSA vocab size = 16382
The Skipgram embedding matrix shape = (16383, 50)
The Skipgram vocab size = 16383


The following code will serve as the sanity check that `vocab_lsa` and `vocab_sg` contain the same words

In [11]:
lsa_word_set = set([item[0] for item in vocab_lsa.items()])
sg_word_set = set([item[0] for item in vocab_sg.items()])
sym_diff = lsa_word_set.symmetric_difference(sg_word_set)

if len(sym_diff) == 0:
    print("vocab_lsa and vocab_sg contain the same words!")
else:
    print("The word that only appear in one vocab: {}".format(sym_diff))

The word that only appear in one vocab: {'</s>'}


If the only word from the `symmetric_difference()` function is `</s>`, then your implementation should be fine. (`</s>` was added by `fasttext` automatically to the end of each text.)

## 3. Evaluation (5 points)

In this homework, we will only use intrinsic evaluation. Specifically, for a list of predefined word pairs with their similarity scores, the evaluation is to calculate the correlation between the predefined similarity scores and the cosine similarity scores based on word embeddings. The higher the correlation, the better the quality of word embeddings.

In [12]:
def load_wordpairs(fname = "embeddings/word-pairs.txt", vocab=None):
    records = {}
    with open(fname) as fin:
        for line in fin:
            items = line.strip().split(",")
            if (items[1] in vocab) and (items[2] in vocab): # make sure both words in the vocab
                records[(items[1],items[2])] = float(items[3])
    print("Load {} pairs of words for evaluation".format(len(records)))
    return records

### 3.1 Word similarity correlation (2 points)

The purpose of this section is to implement the correlation function that compares the predefined scores and the scores computed by cosine similarity. The code of the correlation function is almost done, and the only thing left is the code for computing cosine similarity.

In [13]:
from scipy.stats import pearsonr
# TODO: Add necessary packages here
from sklearn.metrics.pairwise import cosine_similarity

def correlation(records, embeddings, vocab):
    predefined_scores = []
    cossim_scores = []

    for (words, sim_score) in records.items():
        predefined_scores.append(sim_score)
        # ---------------------------------
        # TODO: add your code here for computing the cossine similarity
        #       between words[0] and words[1], and assign the value to variable "score"

        embedding_1 = embeddings[vocab[words[0]]]
        embedding_2 = embeddings[vocab[words[1]]]
        embedding_1 = embedding_1.reshape(1, -1)
        embedding_2 = embedding_2.reshape(1, -1)

        score = cosine_similarity(embedding_1, embedding_2)
        cossim_scores.append(score.item())

        # ---------------------------------

    corr = pearsonr(predefined_scores, cossim_scores)
    return corr

#-----------


Run the following code block to calculate the correlations between pre-defined similarity scores and the cosine similarity scores based on word embeddings

In [14]:
records_lsa = load_wordpairs(vocab=vocab_lsa)
corr_lsa = correlation(records_lsa, embeddings_lsa, vocab_lsa)
print("The correlation with the LSA embeddings = {} with p-value {}".format(corr_lsa[0], corr_lsa[1]))
records_sg = load_wordpairs(vocab=vocab_sg)
corr_sg = correlation(records_sg, embeddings_sg, vocab_sg)
print("The correlation with the LSA embeddings = {} with p-value {}".format(corr_sg[0], corr_sg[1]))

if corr_lsa[0] > corr_sg[0]:
    print("LSA is better than Skip-gram")
elif corr_lsa[0] < corr_sg[0]:
    print("Skipgram is better than LSA")

Load 143 pairs of words for evaluation
The correlation with the LSA embeddings = 0.21193065870544645 with p-value 0.011053229207843563
Load 143 pairs of words for evaluation
The correlation with the LSA embeddings = 0.22302258184809212 with p-value 0.007420400658706261
Skipgram is better than LSA


### 3.2 Analysis of context window size in Skipgram (3 points)

With the correlation function, we can analyze the effect of different context window sizes in the Skipgram model. Specifically, please call the previous implementation

- `Skipgram(fname, ws, dim=50)` with the context window size `ws` as 3, 6, 9, 12, 15
- For each context window size, calculate the correlation using the function `correlation(records, embeddings, vocab)`
- **Print out** the fives correlation scores in your final submission: one score per line with the following format
<center> ws\t correlation</center>

In [15]:
# TODO: add your code here

for ws in [3, 6, 9, 12, 15]:
  embeddings_sg, vocab_sg = Skipgram(ws=ws)
  records_sg = load_wordpairs(vocab=vocab_sg)
  corr_sg = correlation(records_sg, embeddings_sg, vocab_sg)
  print("ws: {} \t corr: {}".format(ws, corr_sg[0]))


Load 143 pairs of words for evaluation
ws: 3 	 corr: 0.22302258184809212
Load 143 pairs of words for evaluation
ws: 6 	 corr: 0.2706073632907414
Load 143 pairs of words for evaluation
ws: 9 	 corr: 0.3450769931789064
Load 143 pairs of words for evaluation
ws: 12 	 corr: 0.3751060485093665
Load 143 pairs of words for evaluation
ws: 15 	 corr: 0.4093035490536317


Similar experiment can also be conducted on the parameter of negative examples `neg`, but it will not be included in this homework.